Reference :  
https://github.com/sleepingcat4/bert-textgeneration  

https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb

## Import the libraries

In [1]:
!pip install datasets
!pip install accelerate
!pip install sentencepiece
!pip install --upgrade huggingface_hub
!pip -q install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none

In [1]:
from huggingface_hub import login
login()

In [2]:
#from google.colab import userdata
#userdata.get('HF_TOKEN')

'huanghe'

In [1]:
from transformers import CamembertForMaskedLM, AutoTokenizer, AutoConfig

In [2]:
camembert = CamembertForMaskedLM.from_pretrained('camembert-base')

/home/huang_h/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
tokenizer = AutoTokenizer.from_pretrained('camembert-base')

## Load CSV File

View [Source](https://stackoverflow.com/questions/76001128/splitting-dataset-into-train-test-and-validation-using-huggingface-datasets-fun)

In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from datasets import *

data_path = 'exos_to_csv.csv'
dataset = load_dataset('csv', data_files=data_path)

In [5]:
# Split the dataset into 3 sets for train, test and validation
train_testsplit = dataset['train'].train_test_split(test_size=0.2)
test_validsplit = train_testsplit['test'].train_test_split(test_size=0.5)

In [18]:
ds = DatasetDict({
    'train': train_testsplit['train'],
    'test': test_validsplit['test'],
    'valid': test_validsplit['train']
})

In [19]:
ds

DatasetDict({
    train: Dataset({
        features: ['filename', 'consigne', 'enonce'],
        num_rows: 1022
    })
    test: Dataset({
        features: ['filename', 'consigne', 'enonce'],
        num_rows: 128
    })
    valid: Dataset({
        features: ['filename', 'consigne', 'enonce'],
        num_rows: 128
    })
})

Create a random sample

In [20]:
sample = ds["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Enonce: {row['enonce']}'")


'>>> Enonce: a. saucisse ◆ commande ◆ différent ◆ brosse ◆ irrégulier ◆ étonné ◆ accent ◆ attaque ◆ flotte
b. maisonnette ◆ coccinelle ◆ messager ◆ grammatical ◆ pellicule ◆ irresponsabilité'

'>>> Enonce: a. une camarade ◆ une peintre ◆ une violoniste ◆ une artiste
b. une pâtissière ◆ une caissière ◆ une crémière ◆ une fermière
c. une chirurgienne ◆ une magicienne ◆ une musicienne ◆ une mécanicienne
d. une chanteuse ◆ une acheteuse ◆ une parfumeuse ◆ une joueuse'

'>>> Enonce: a. Résultat d’une addition.
b. Le plomb, l’acier, le cuivre appartiennent à cette catégorie.
c. Mes parents le lisent tous les matins.
d. Ce jour-là, on se déguise.'


For both auto-regressive and masked language modeling, a common preprocessing step is to concatenate all the examples and then split the whole corpus into chunks of equal size.

In [21]:
def tokenize_function(examples):
    result = tokenizer(examples["enonce"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

In [27]:
# Use batched=True to activate fast multithreading!
tokenized_datasets = ds.map(
    tokenize_function, batched=True, remove_columns=["filename", "consigne", "enonce"]
)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 1022
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 128
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 128
    })
})

Inspecting the `model_max_length` attribute of the tokenizer:

In [28]:
tokenizer.model_max_length

512

We set `chunk_size`.

In [29]:
chunk_size = 32

In [30]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Enonce {idx} length: {len(sample)}'")

'>>> Enonce 0 length: 32'
'>>> Enonce 1 length: 26'
'>>> Enonce 2 length: 55'


In [31]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated length: {total_length}'")

'>>> Concatenated length: 113'


In [32]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}
### Example concatenated_examples = {"input_ids": [101, 1029, 2048, ...], "attention_mask": [...], ...}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 32'
'>>> Chunk length: 32'
'>>> Chunk length: 32'
'>>> Chunk length: 17'


In [33]:
def chunk_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]]) # If the first key is 'input_ids', we take its length
    # We drop the last chunk if it's smaller than chunk_size, if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size # To make sure that the total length is a multiple of chunk_size
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [34]:
lm_datasets = tokenized_datasets.map(chunk_texts, batched=True)
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1695
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 220
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 215
    })
})

In [134]:
import re

def mask_last(exemples):
  batch_sentences = []
  
  for lm_texts in exemples["input_ids"]:
    decode_texts = tokenizer.decode(lm_texts)

    decode_texts = decode_texts[:-1] + '<mask>'
    batch_sentences.append(decode_texts)
  
  return {'masked': batch_sentences}



In [135]:
decode = lm_datasets.map(mask_last, batched=True)

Map:   0%|          | 0/1695 [00:00<?, ? examples/s]

Map:   0%|          | 0/220 [00:00<?, ? examples/s]

Map:   0%|          | 0/215 [00:00<?, ? examples/s]

In [137]:
decode["train"][1]["masked"]

'<s> fleur <unk> dent <unk> charcuterie <unk> écrire <unk> parfumer <unk> dessiner <unk> vendre <unk> enseigner</s><s> a. le vocabulaire <mask>'

In [138]:
MASK_TOKEN_ID = tokenizer.mask_token_id

In [142]:
def mask_labels(result):

    result['labels'] = [[] for _ in result['input_ids']]  
    # Mask the last token of each chunk directly
    for i, chunk in enumerate(result['input_ids']):
        last_index = len(chunk) - 1 # last token index
        original_token_id = chunk[last_index] 
        chunk[last_index] = MASK_TOKEN_ID # last masked token id

        labels = [-100] * len(chunk)
        labels[last_index] = original_token_id

        result["labels"][i] = labels

    return result

In [143]:
masked_datasets = decode.map(mask_labels, batched=True)

Map:   0%|          | 0/1695 [00:00<?, ? examples/s]

Map:   0%|          | 0/220 [00:00<?, ? examples/s]

Map:   0%|          | 0/215 [00:00<?, ? examples/s]

In [144]:
tokenizer.decode(masked_datasets["train"][1]["input_ids"])

'<s> fleur <unk> dent <unk> charcuterie <unk> écrire <unk> parfumer <unk> dessiner <unk> vendre <unk> enseigner</s><s> a. le vocabulaire<mask>'

In [63]:
masked_datasets = masked_datasets.remove_columns(["word_ids"])

# Prepare Training Procedure

In [64]:
import torch
torch.cuda.set_device(1)
torch.cuda.current_device()

1

In [65]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer)

In [66]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
from accelerate import Accelerator
from transformers import get_scheduler
from transformers import default_data_collator


batch_size = 8

# load the train dataset for traing
train_dataloader = DataLoader(masked_datasets["train"], shuffle=True, batch_size=batch_size, collate_fn=data_collator)

# load the test dataset for evaluation
eval_dataloader = DataLoader(masked_datasets["valid"], batch_size=batch_size, collate_fn=default_data_collator)

# initialize pretrained bert model
model = camembert

# set the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# initialize accelerator for training
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

# set the number of epochs which is set to 30
num_train_epochs = 10
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

# define the learning rate scheduler for training
lr_scheduler = get_scheduler("linear",optimizer=optimizer,num_warmup_steps=0,num_training_steps=num_training_steps)

In [67]:
import math
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

output_dir = "camemBERT_finetuning"

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataloader)]

    # perplexity metric used for mask language model training
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")
    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

    # Save model
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)

  0%|          | 0/2160 [00:00<?, ?it/s]

>>> Epoch 0: Perplexity: 106.7709374380134
>>> Epoch 1: Perplexity: 90.50304454453625
>>> Epoch 2: Perplexity: 101.96873166557823
>>> Epoch 3: Perplexity: 74.13096599599437
>>> Epoch 4: Perplexity: 50.65498378767439
>>> Epoch 5: Perplexity: 54.43193941705431
>>> Epoch 6: Perplexity: 43.236859198609274
>>> Epoch 7: Perplexity: 61.74856379505376
>>> Epoch 8: Perplexity: 53.405149867696416
>>> Epoch 9: Perplexity: 56.21801574319338


# Test Model

In [102]:
from transformers import pipeline

pred_model = pipeline("fill-mask", model = "camemBERT_finetuning")

text = "Recopie ce texte en choisissant le signe de ponctuation qui <mask>."

preds = pred_model(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> Recopie ce texte en choisissant le signe de ponctuation qui....
>>> Recopie ce texte en choisissant le signe de ponctuation qui (...).
>>> Recopie ce texte en choisissant le signe de ponctuation qui....
>>> Recopie ce texte en choisissant le signe de ponctuation qui [...].
>>> Recopie ce texte en choisissant le signe de ponctuation qui suit.


In [72]:
model.push_to_hub("huanghe/camemBERT")

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/huanghe/camemBERT/commit/ba86b6800b0d2ddc75361f30359786d8aff0a925', commit_message='Upload CamembertForMaskedLM', commit_description='', oid='ba86b6800b0d2ddc75361f30359786d8aff0a925', pr_url=None, pr_revision=None, pr_num=None)

In [73]:
tokenizer.push_to_hub("huanghe/camemBERT")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/huanghe/camemBERT/commit/286e1bb005a094284c9234dde78f7c8148ec8c7f', commit_message='Upload tokenizer', commit_description='', oid='286e1bb005a094284c9234dde78f7c8148ec8c7f', pr_url=None, pr_revision=None, pr_num=None)

In [74]:
from transformers import AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("huanghe/camemBERT")

model = AutoModelForMaskedLM.from_pretrained("huanghe/camemBERT") 

inputs = tokenizer(text, return_tensors="pt")


with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
output = tokenizer.decode(predicted_token_id)
print(output) 

/home/huang_h/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/374 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

...


In [99]:
text += " " + "<mask>"

for _ in range(10):
    inputs = tokenizer(text, return_tensors="pt")
    
    # Get the logits from the model
    with torch.no_grad():
        token_logits = model(**inputs).logits

   # Find the MASK token index and get its logits
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
    mask_token_logits = token_logits[0, mask_token_index, :]

    # Pick the top token from the logits
    top_token = torch.topk(mask_token_logits, 1, dim=1).indices[0].tolist()[0]
    top_token_word = tokenizer.decode([top_token])

    # Replace MASK with the top token and add a new MASK at the end
    text = text.replace(tokenizer.mask_token, top_token_word + " " + "<mask>")

# Display the final text
print(text)

    




Recopie ce texte en choisissant le signe de ponctuation qui est : le mot . Le mot magique de la lettre muette muette muette muette muette muette muette muette muette <mask>.... est : le mot . Le mot magique de la lettre muette muette muette muette muette muette muette muette muette <mask>.... est : le mot . Le mot magique de la lettre muette muette muette muette muette muette muette muette muette <mask> est : le mot . Le mot magique de la lettre muette muette muette muette muette muette muette muette muette <mask> lettre muette muette muette muette muette muette muette muette muette <mask>
